In [6]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from datetime import datetime
import itertools
from sklearn.model_selection import TimeSeriesSplit
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import acquire
import prepare

- Using your store items data you prepped in lesson 2 exercises:

In [3]:
df = prepare.prepare_sale()

In [5]:
df.head(3)

,item_id,sale_amount,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode,month,day_of_week,total_sales
sale_date,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,1,13.0,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,01-Jan,2-Tuesday,10.92
2013-01-01 00:00:00+00:00,17,26.0,295813,3,Ducal,Ducal Refried Red Beans,1.16,88313590791,88313590791,2118 Fredericksburg Rdj,San Antonio,TX,78201,01-Jan,2-Tuesday,30.16
2013-01-01 00:00:00+00:00,7,32.0,125995,10,Twinings Of London,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004,8503 NW Military Hwy,San Antonio,TX,78231,01-Jan,2-Tuesday,308.48


1. Split your data into train and test using the sklearn.model_selection.TimeSeriesSplit method.

2. Validate your splits by plotting X_train and y_train.

3. Plot the weekly average & the 7-day moving average. Compare the 2 plots.

4. Plot the daily difference. Observe whether usage seems to vary drastically from day to day or has more of a smooth transition.

5. Plot a time series decomposition.

6. Create a lag plot (day over day).

7. Run a lag correlation.

- Using your OPS data you prepped in lesson 2 exercises:

In [7]:
ops = prepare.prepare_ops()

In [8]:
ops.head(3)

,Consumption,Wind,Solar,Wind+Solar,month,year
Date,,,,,,
2006-01-01,1069.184,NaN,NaN,NaN,1,2006
2006-01-02,1380.521,NaN,NaN,NaN,1,2006
2006-01-03,1442.533,NaN,NaN,NaN,1,2006


1. Split your data into train and test using the percent cutoff method.

2. Validate your splits by plotting X_train and y_train.

3. Plot the weekly average & the 7-day moving average. Compare the 2 plots.

4. Group the electricity consumption time series by month of year, to explore annual seasonality.

5. Plot the daily difference. Observe whether usage seems to vary drastically from day to day or has more of a smooth transition.

6. Plot a time series decomposition. Takeaways?